In [14]:
from pymongo.mongo_client import MongoClient

client = MongoClient(host="localhost", port=27017)
job_col = client.Cacher.Job
client_col = client.Cacher.Client

In [2]:
jobInfo = [
    {
        "meta": {
            "username": "zhuangwei",
            "jobId": "j1"
        },
        "QoS": {
            "useCache": True
        },
        "policy": {
            "chunkSize": 1024,
            "chunks": [{"key": 1, "location": "redis"}, {"key": 2, "location": "disk"}]
        }
    },
    {
        "meta": {
            "username": "zhuangwei",
            "jobId": "j2"
        },
        "QoS": {
            "useCache": True
        },
        "policy": {
            "chunkSize": 1024,
            "chunks": [{"key": 4, "location": "redis"}, {"key": 5, "location": "disk"}]
        }
    },
    {
        "meta": {
            "username": "zhuangwei",
            "jobId": "j3"
        },
        "QoS": {
            "useCache": True
        },
        "policy": {
            "chunkSize": 1024,
            "chunks": [{"key": 3, "location": "redis"}, {"key": 6, "location": "disk"}]
        }
    }
]

In [16]:
job_col.insert_many(jobInfo)

In [179]:
result = job_col.aggregate([
    {"$unwind": "$policy.chunks"},
    {"$project": {"_id": 0, "key": "$policy.chunks.key",
        "chunks": {"$regexFind": {"input": "$policy.chunks.location", "regex": "^redis", "options": "m"}}}},
    {"$match": {"chunks": {"$ne": None}}},
    {"$sort": {"key": 1}},
    {"$limit": 2},
    {"$project": {"key": 1}},
    {"$group": {"_id": "$_id", "keys": {"$push": "$key"}}}
])

# print(list(result))
for chunk in result:
    print(chunk)

{'_id': None, 'keys': [1, 3]}


In [12]:
resp = job_col.find(filter={"$and": [{"meta.username": 'zhuangwei'}, {"meta.jobId": 'j1'}]})
for obj in resp:
    print(obj)

{'_id': ObjectId('62a3c36081267a5663a988bc'), 'meta': {'username': 'zhuangwei', 'jobId': 'j1'}, 'QoS': {'useCache': True}, 'policy': {'chunkSize': 1024, 'chunks': [{'key': 1, 'location': 'redis'}, {'key': 2, 'location': 'disk'}]}}


In [19]:
resp = job_col.find({"policy.chunks": {"$elemMatch": {"key": 3}}})
list(resp)

[{'_id': ObjectId('62a3c36081267a5663a988be'),
  'meta': {'username': 'zhuangwei', 'jobId': 'j3'},
  'QoS': {'useCache': True},
  'policy': {'chunkSize': 1024,
   'chunks': [{'key': 3, 'location': 'redis'},
    {'key': 6, 'location': 'disk'}]}}]

In [92]:
resp = job_col.aggregate([
    {"$project": {
        "_id": 0, 
        "chunk": {
            "$filter": {
                "input": "$policy.chunks", 
                "as": "chunks", 
                "cond": {"$eq": ["$$chunks.key", 2]}}
            }
    }},
    {"$unwind": "$chunk"}
]).next()['chunk']
list(resp)

[{'chunk': {'key': 2, 'location': 'disk'}}]

In [15]:
client_info = [
    {
        "username": "user1",
        "jobs": ["job1", 'job2']
    },
    {
        "username": "user2",
        "jobs": ["job3", 'job4']
    }
]

In [16]:
client_col.insert_many(client_info)